# 8 数据规整：连接、联合与重塑
重点关注数据联合、连接以及重排

## 8.1 分层索引

In [1]:
import pandas as pd
import numpy as np

In [2]:
# 双层索引的Series
data = pd.Series(np.random.randn(9),
                 index=[['a', 'a', 'a', 'b', 'b', 'c', 'c', 'd', 'd'],
                        [1, 2, 3, 1, 3, 1, 2, 2, 3]])
data

a  1    0.737238
   2   -0.931812
   3   -0.130993
b  1   -1.167149
   3   -0.166761
c  1    0.283305
   2    2.316861
d  2    1.008046
   3    1.943403
dtype: float64

In [3]:
data.index

MultiIndex([('a', 1),
            ('a', 2),
            ('a', 3),
            ('b', 1),
            ('b', 3),
            ('c', 1),
            ('c', 2),
            ('d', 2),
            ('d', 3)],
           )

In [4]:
# 使用分层索引可以简洁地选择出数据的子集
data['b']

1   -1.167149
3   -0.166761
dtype: float64

In [5]:
data['b':'c']

b  1   -1.167149
   3   -0.166761
c  1    0.283305
   2    2.316861
dtype: float64

In [6]:
data.loc[['b', 'd']]

b  1   -1.167149
   3   -0.166761
d  2    1.008046
   3    1.943403
dtype: float64

In [7]:
# 内部层级中也可以选择，比如选择内层中索引为2的
data.loc[:, 2]

a   -0.931812
c    2.316861
d    1.008046
dtype: float64

In [8]:
# 重塑数据
data.unstack()

,1,2,3
a,0.737238,-0.931812,-0.130993
b,-1.167149,NaN,-0.166761
c,0.283305,2.316861,NaN
d,NaN,1.008046,1.943403


In [9]:
# 在DataFrame中，每个轴都可以有分层索引
df = pd.DataFrame(np.arange(12).reshape(4, 3),
                  index=[['a', 'a', 'b', 'b'], [1, 2, 1, 2]],
                  columns=[['Ohio', 'Ohio', 'California'], ['Green', 'Red', 'Green']])
df.index.names = ['key1', 'key2']
df.columns.names = ['State', 'Color']
df

State      Ohio     California
Color     Green Red      Green
key1 key2                     
a    1        0   1          2
     2        3   4          5
b    1        6   7          8
     2        9  10         11

In [10]:
# 筛选出数据
df['Ohio']

Color      Green  Red
key1 key2            
a    1         0    1
     2         3    4
b    1         6    7
     2         9   10

In [11]:
df.loc['a']

State  Ohio     California
Color Green Red      Green
key2                      
1         0   1          2
2         3   4          5

In [12]:
from pandas import MultiIndex

columns = MultiIndex.from_arrays([['Ohio', 'Ohio', 'California'], ['Green', 'Red', 'Green']],
                                 names=['State', 'Color'])
indexes = MultiIndex.from_arrays([['a', 'a', 'b', 'b'], [1, 2, 1, 2]],
                                 names=['key1', 'key2'])

In [13]:
df2 = pd.DataFrame(np.arange(12).reshape(4, 3),
                   columns=columns,
                   index=indexes)

In [14]:
df2

State      Ohio     California
Color     Green Red      Green
key1 key2                     
a    1        0   1          2
     2        3   4          5
b    1        6   7          8
     2        9  10         11

### 8.1.1 重排序和层级排序

In [15]:
# 重新排列轴上的层级顺序,
# swaplevel接收两个层级序号或层级名称，返回一个层级变更的新对象
df2.swaplevel('key1', 'key2')

State      Ohio     California
Color     Green Red      Green
key2 key1                     
1    a        0   1          2
2    a        3   4          5
1    b        6   7          8
2    b        9  10         11

In [16]:
# level=0表示第一层(key1)，level=1表示第二层(key2)
# 如果索引按照字典顺序从最外层开始排序，那么数据选择性能会更好
df2.sort_index(level=0, ascending=False)

State      Ohio     California
Color     Green Red      Green
key1 key2                     
b    2        9  10         11
     1        6   7          8
a    2        3   4          5
     1        0   1          2

### 8.1.2 按层级进行汇总统计
Series和DataFrame上很多描述性和统计行统计有一个level选项，通过level可以指定要在某个特定的轴上进行聚合

In [17]:
df2.groupby(level='key1').sum()

State  Ohio     California
Color Green Red      Green
key1                      
a         3   5          7
b        15  17         19

In [18]:
df2.groupby(level='Color', axis=1).sum()

Color      Green  Red
key1 key2            
a    1         2    1
     2         8    4
b    1        14    7
     2        20   10

### 8.1.3 使用DataFrame的列进行索引

In [19]:
df3 = pd.DataFrame({'a': range(7),
                    'b': range(7, 0, -1),
                    'c': ['one', 'one', 'one', 'two', 'two', 'two', 'two'],
                    'd': [0, 1, 2, 0, 1, 2, 3]})
df3

,a,b,c,d
0,0,7,one,0
1,1,6,one,1
2,2,5,one,2
3,3,4,two,0
4,4,3,two,1
5,5,2,two,2
6,6,1,two,3


In [20]:
df3.set_index(['c', 'd'])

a  b
c   d      
one 0  0  7
    1  1  6
    2  2  5
two 0  3  4
    1  4  3
    2  5  2
    3  6  1

In [21]:
# 默认情况下，设置为index的列会从DataFrame中移除，也可以保留
df3.set_index(['c', 'd'], drop=False)

a  b    c  d
c   d              
one 0  0  7  one  0
    1  1  6  one  1
    2  2  5  one  2
two 0  3  4  two  0
    1  4  3  two  1
    2  5  2  two  2
    3  6  1  two  3

In [22]:
# reset_index是set_index的反向操作
df4 = df3.set_index(['c', 'd'])
df4.reset_index()

,c,d,a,b
0,one,0,0,7
1,one,1,1,6
2,one,2,2,5
3,two,0,3,4
4,two,1,4,3
5,two,2,5,2
6,two,3,6,1


## 8.2 联合与合并数据集

In [23]:
# 数据库风格的DataFrame连接
df1 = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
                    'data1': range(7)})
df2 = pd.DataFrame({'key': ['a', 'b', 'd'],
                    'data2': range(3)})
df1

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


In [24]:
df2

,key,data2
0,a,0
1,b,1
2,d,2


In [25]:
# 使用pd.merge连接
pd.merge(df1, df2, on='key')

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


In [26]:
# 如果列名不同，可以手动指定连接列
df3 = pd.DataFrame({'lkey': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
                    'data1': range(7)})
df4 = pd.DataFrame({'rkey': ['a', 'b', 'd'],
                    'data2': range(3)})
pd.merge(df3, df4, left_on='lkey', right_on='rkey')

,lkey,data1,rkey,data2
0,b,0,b,1
1,b,1,b,1
2,b,6,b,1
3,a,2,a,0
4,a,4,a,0
5,a,5,a,0


In [27]:
# pd.merge默认情况下是内连接，取交集
# 可以指定连接方式 outer为外连接，取并集
pd.merge(df3, df4, how='outer', left_on='lkey', right_on='rkey')

,lkey,data1,rkey,data2
0,b,0.0,b,1.0
1,b,1.0,b,1.0
2,b,6.0,b,1.0
3,a,2.0,a,0.0
4,a,4.0,a,0.0
5,a,5.0,a,0.0
6,c,3.0,NaN,NaN
7,NaN,NaN,d,2.0


In [28]:
# 使用多键合并时，传入一个列名的列表
left = pd.DataFrame({'key1': ['foo', 'foo', 'bar'],
                     'key2': ['one', 'two', 'one'],
                     'lval': [1, 2, 3]})
right = pd.DataFrame({'key1': ['foo', 'foo', 'bar', 'bar'],
                      'key2': ['one', 'one', 'one', 'two'],
                      'lval': [4, 5, 6, 7]})
pd.merge(left, right, on=['key1', 'key2'], how='outer')

,key1,key2,lval_x,lval_y
0,foo,one,1.0,4.0
1,foo,one,1.0,5.0
2,foo,two,2.0,NaN
3,bar,one,3.0,6.0
4,bar,two,NaN,7.0


In [29]:
# 处理重叠的列名，使用suffixes参数
pd.merge(left, right, on='key1', suffixes=('_left', '_right'))

,key1,key2_left,lval_left,key2_right,lval_right
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


### 8.2.2 根据索引合并

In [30]:
left1 = pd.DataFrame({'key': ['a', 'b', 'a', 'a', 'b', 'c'],
                      'value': range(6)})
right1 = pd.DataFrame({'group_val': [3.5, 7]},
                      index=['a', 'b'])
# left_index=True或者right_index=True表示需要用来作为合并的键
pd.merge(left1, right1, left_on='key', right_index=True)

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0


In [31]:
pd.merge(left1, right1, left_on='key', right_index=True, how='outer')

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0
5,c,5,NaN


In [32]:
# 在多层索引的情况下，事情会变的更复杂，在索引上的连接是一个隐式的多键合并
df1 = pd.DataFrame({'key1': ['Ohio', 'Ohio', 'Ohio', 'Nevada', 'Nevada'],
                    'key2': [2000, 2001, 2002, 2001, 2002],
                    'data': np.arange(5.)})
df2 = pd.DataFrame(np.arange(12).reshape(6, 2),
                   index=[['Nevada', 'Nevada', 'Ohio', 'Ohio', 'Ohio', 'Ohio'],
                          [2001, 2000, 2000, 2000, 2001, 2002]])
df1

,key1,key2,data
0,Ohio,2000,0.0
1,Ohio,2001,1.0
2,Ohio,2002,2.0
3,Nevada,2001,3.0
4,Nevada,2002,4.0


In [33]:
df2

0   1
Nevada 2001   0   1
       2000   2   3
Ohio   2000   4   5
       2000   6   7
       2001   8   9
       2002  10  11

In [34]:
# 对于多重索引合并的情况，需要以列表的形式指明合并所需的多个列
pd.merge(df1, df2, left_on=['key1', 'key2'], right_index=True, how='outer')

,key1,key2,data,0,1
0,Ohio,2000,0.0,4.0,5.0
0,Ohio,2000,0.0,6.0,7.0
1,Ohio,2001,1.0,8.0,9.0
2,Ohio,2002,2.0,10.0,11.0
3,Nevada,2001,3.0,0.0,1.0
4,Nevada,2002,4.0,NaN,NaN
4,Nevada,2000,NaN,2.0,3.0


In [35]:
# 使用两边的索引合并也是可以的
df3 = pd.DataFrame([[1, 2], [3, 4], [5, 6]],
                   index=['a', 'c', 'e'],
                   columns=['Ohio', 'Nevada'])
df4 = pd.DataFrame([[7, 8], [9, 10], [11, 12], [13, 14]],
                   index=['b', 'c', 'd', 'e'],
                   columns=['Missouri', 'Alabama'])
df3

,Ohio,Nevada
a,1,2
c,3,4
e,5,6


In [36]:
df4

,Missouri,Alabama
b,7,8
c,9,10
d,11,12
e,13,14


In [37]:
pd.merge(df3, df4, how='outer', left_index=True, right_index=True)

,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0


In [38]:
# 也可以使用join方法
df3.join(df4, how='outer')

,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0


In [39]:
df3.join(df4, how='left')

,Ohio,Nevada,Missouri,Alabama
a,1,2,NaN,NaN
c,3,4,9.0,10.0
e,5,6,13.0,14.0


In [40]:
another = pd.DataFrame([[7, 8], [9, 10], [11, 12], [16, 17]],
                       index=['a', 'c', 'e', 'f'],
                       columns=['New York', 'Oregon'])
another

,New York,Oregon
a,7,8
c,9,10
e,11,12
f,16,17


In [41]:
df3.join([df4, another])

,Ohio,Nevada,Missouri,Alabama,New York,Oregon
a,1.0,2.0,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0,9.0,10.0
e,5.0,6.0,13.0,14.0,11.0,12.0


In [42]:
df3.join([df4, another], how='outer')

,Ohio,Nevada,Missouri,Alabama,New York,Oregon
a,1.0,2.0,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0,9.0,10.0
e,5.0,6.0,13.0,14.0,11.0,12.0
b,NaN,NaN,7.0,8.0,NaN,NaN
d,NaN,NaN,11.0,12.0,NaN,NaN
f,NaN,NaN,NaN,NaN,16.0,17.0


In [43]:
pd.merge(df3, df4, left_index=True, right_index=True)

,Ohio,Nevada,Missouri,Alabama
c,3,4,9,10
e,5,6,13,14


### 8.2.3 沿轴向连接

In [44]:
# 拼接、绑定或堆叠
arr = np.arange(12).reshape(4, 3)
arr

array([[ 0,  1,  2],
       [ 3,  4,  5],
       [ 6,  7,  8],
       [ 9, 10, 11]])

In [45]:
# Numpy的拼接方法接收一个数组组成的列表
np.concatenate([arr, arr], axis=1)

array([[ 0,  1,  2,  0,  1,  2],
       [ 3,  4,  5,  3,  4,  5],
       [ 6,  7,  8,  6,  7,  8],
       [ 9, 10, 11,  9, 10, 11]])

In [46]:
# pandas的concat函数
s1 = pd.Series([0, 1], index=['a', 'b'])
s2 = pd.Series([2, 3, 4], index=['c', 'd', 'e'])
s3 = pd.Series([5, 7], index=['f', 'g'])
pd.concat([s1, s2, s3])  # 传入的对象是可迭代的，默认在0轴上进行连接

a    0
b    1
c    2
d    3
e    4
f    5
g    7
dtype: int64

In [47]:
pd.concat((s1, s2, s3), axis=1)

,0,1,2
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,7.0


In [48]:
# 假如想在连接上创建多层索引
# 添加的外层轴索引是和s1,s2,s3相关联的
pd.concat([s1, s2, s3], keys=['one', 'two', 'three'])

one    a    0
       b    1
two    c    2
       d    3
       e    4
three  f    5
       g    7
dtype: int64

In [49]:
# 当沿着axis=1上添加时，就成了DataFrame的列头
pd.concat([s1, s2, s3], keys=['one', 'two', 'three'], axis=1)

,one,two,three
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,7.0


In [50]:
# 相同的逻辑使用在DataFrame上
df1 = pd.DataFrame(np.arange(6).reshape(3, 2),
                   index=list('abc'),
                   columns=['one', 'two'])
df2 = pd.DataFrame(5 + np.arange(4).reshape(2, 2),
                   index=list('ac'),
                   columns=['three', 'four'])
df1

,one,two
a,0,1
b,2,3
c,4,5


In [51]:
df2

,three,four
a,5,6
c,7,8


In [52]:
pd.concat([df1, df2], axis=1, keys=['l1', 'l2'])

l1        l2     
  one two three four
a   0   1   5.0  6.0
b   2   3   NaN  NaN
c   4   5   7.0  8.0

In [53]:
# 如果传递的对象是字典而不是列表，则字典的键会用于keys选项
pd.concat({'l1': df1, 'l2': df2}, axis=1)

l1        l2     
  one two three four
a   0   1   5.0  6.0
b   2   3   NaN  NaN
c   4   5   7.0  8.0

In [54]:
# 还有一些额外的参数负责多层索引生成
pd.concat([df1, df2], axis=1, keys=['l1', 'l2'], names=['upper', 'lower'])

upper  l1        l2     
lower one two three four
a       0   1   5.0  6.0
b       2   3   NaN  NaN
c       4   5   7.0  8.0

In [55]:
# 最后需要考虑的是行索引中不包含任何相关数据的DataFrame
df1 = pd.DataFrame(np.random.randn(3, 4),
                   columns=list('abcd'))
df2 = pd.DataFrame(np.random.randn(2, 3),
                   columns=list('bda'))
pd.concat([df1, df2], ignore_index=True)  # 忽略掉index，就是直接合并

,a,b,c,d
0,3.117151,0.793419,0.068955,0.557880
1,0.480597,-1.180502,-0.595152,-0.147631
2,0.710013,-0.120387,-0.200739,-0.189948
3,0.340164,0.842555,NaN,-0.703790
4,1.492002,0.552227,NaN,0.437791


### 8.2.4 联合重叠数据

In [56]:
a = pd.Series([np.nan, 2.5, 0, 3.5, 4.5, np.nan],
              index=list('fedcba'))
b = pd.Series([0, np.nan, 2, np.nan, np.nan, 5],
              index=list('abcdef'))
a

f    NaN
e    2.5
d    0.0
c    3.5
b    4.5
a    NaN
dtype: float64

In [57]:
b

a    0.0
b    NaN
c    2.0
d    NaN
e    NaN
f    5.0
dtype: float64

In [58]:
# Series的combine_first函数就是去两者索引相同时出现的第一个值
b.combine_first(a)

a    0.0
b    4.5
c    2.0
d    0.0
e    2.5
f    5.0
dtype: float64

## 8.3 重塑和透视

### 8.3.1 使用多层索引进行重塑

In [59]:
# stack: 用于将列中数据透视到行
# unstack: 用于将行数据透视到列
data = pd.DataFrame(np.arange(6).reshape(2, 3),
                    index=pd.Index(['Ohio', 'Colorado'], name='state'),
                    columns=pd.Index(['one', 'two', 'three'], name='number'))
data

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


In [60]:
data.stack()

state     number
Ohio      one       0
          two       1
          three     2
Colorado  one       3
          two       4
          three     5
dtype: int64

In [61]:
data.unstack()

number  state   
one     Ohio        0
        Colorado    3
two     Ohio        1
        Colorado    4
three   Ohio        2
        Colorado    5
dtype: int64

In [62]:
b = data.unstack()
type(b)

pandas.core.series.Series

In [63]:
# stack和unstack可以将DataFrame转化为Series
# 对Series执行unstack，可以转化为DataFrame
c = data.stack().unstack()  # 使用一对操作可将数据还原
type(c)

pandas.core.frame.DataFrame

In [64]:
# 如果层级汇总的所有值并未包含于每个子分组是，拆分可能会引入缺失值
s1 = pd.Series([0, 1, 2, 3], index=list('abcd'))
s2 = pd.Series([4, 5, 6], index=list('cde'))
data2 = pd.concat([s1, s2], keys=['one', 'two'])
data2

one  a    0
     b    1
     c    2
     d    3
two  c    4
     d    5
     e    6
dtype: int64

In [65]:
data2.unstack()

,a,b,c,d,e
one,0.0,1.0,2.0,3.0,NaN
two,NaN,NaN,4.0,5.0,6.0


In [66]:
data2.unstack().stack()

one  a    0.0
     b    1.0
     c    2.0
     d    3.0
two  c    4.0
     d    5.0
     e    6.0
dtype: float64

In [67]:
df

State      Ohio     California
Color     Green Red      Green
key1 key2                     
a    1        0   1          2
     2        3   4          5
b    1        6   7          8
     2        9  10         11

### 8.3.2 将"长"透视为"宽"

In [68]:
# 在csv中存储时间序列的方式就是所谓的长格式或堆叠格式
data = pd.read_csv("./examples/macrodata.csv")
data.fillna(0)
data.head()

,year,quarter,realgdp,realcons,realinv,realgovt,realdpi,cpi,m1,tbilrate,unemp,pop,infl,realint
0,1959.0,1.0,2710.349,1707.4,286.898,470.045,1886.9,28.98,139.7,2.82,5.8,177.146,0.00,0.00
1,1959.0,2.0,2778.801,1733.7,310.859,481.301,1919.7,29.15,141.7,3.08,5.1,177.830,2.34,0.74
2,1959.0,3.0,2775.488,1751.8,289.226,491.260,1916.4,29.35,140.5,3.82,5.3,178.657,2.74,1.09
3,1959.0,4.0,2785.204,1753.7,299.356,484.052,1931.3,29.37,140.0,4.33,5.6,179.386,0.27,4.06
4,1960.0,1.0,2847.699,1770.5,331.722,462.199,1955.5,29.54,139.6,3.50,5.2,180.007,2.31,1.19


In [69]:
periods = pd.PeriodIndex(year=data.year, quarter=data.quarter, name='date')
columns = pd.Index(['realgdp', 'infl', 'unemp'], name='item')
# 只取部分列数据
data1 = data.reindex(columns=columns)
# 将periods转换为时间戳格式
data1.index = periods.to_timestamp('D', 'end')
data1.head()

item,realgdp,infl,unemp
date,,,
1959-03-31 23:59:59.999999999,2710.349,0.00,5.8
1959-06-30 23:59:59.999999999,2778.801,2.34,5.1
1959-09-30 23:59:59.999999999,2775.488,2.74,5.3
1959-12-31 23:59:59.999999999,2785.204,0.27,5.6
1960-03-31 23:59:59.999999999,2847.699,2.31,5.2


In [70]:
ldata = data1.stack().reset_index().rename(columns={0: 'value'})
ldata[:10]

,date,item,value
0,1959-03-31 23:59:59.999999999,realgdp,2710.349
1,1959-03-31 23:59:59.999999999,infl,0.000
2,1959-03-31 23:59:59.999999999,unemp,5.800
3,1959-06-30 23:59:59.999999999,realgdp,2778.801
4,1959-06-30 23:59:59.999999999,infl,2.340
5,1959-06-30 23:59:59.999999999,unemp,5.100
6,1959-09-30 23:59:59.999999999,realgdp,2775.488
7,1959-09-30 23:59:59.999999999,infl,2.740
8,1959-09-30 23:59:59.999999999,unemp,5.300
9,1959-12-31 23:59:59.999999999,realgdp,2785.204


In [71]:
# 在关系数据库中，倾向于将data和item作为主键，但是在某些情况下，处理这种格式的数据更为困难
# 更希望同一个时间戳单独成一行，每个不同的item单独一列，也就是转回原始的data格式
# 可以使用DataFrame的pivot方法
# date和item字段分别作为行和列的索引，然后是可选的数值列以填充DataFrame
pivoted = ldata.pivot('date', 'item', 'value')
pivoted

item,infl,realgdp,unemp
date,,,
1959-03-31 23:59:59.999999999,0.00,2710.349,5.8
1959-06-30 23:59:59.999999999,2.34,2778.801,5.1
1959-09-30 23:59:59.999999999,2.74,2775.488,5.3
1959-12-31 23:59:59.999999999,0.27,2785.204,5.6
1960-03-31 23:59:59.999999999,2.31,2847.699,5.2
...,...,...,...
2008-09-30 23:59:59.999999999,-3.16,13324.600,6.0
2008-12-31 23:59:59.999999999,-8.79,13141.920,6.9
2009-03-31 23:59:59.999999999,0.94,12925.410,8.1


In [72]:
# 假如有两个数值列，同时又想重塑
ldata['value2'] = np.random.randn(len(ldata))
ldata[:10]

,date,item,value,value2
0,1959-03-31 23:59:59.999999999,realgdp,2710.349,-0.350004
1,1959-03-31 23:59:59.999999999,infl,0.000,0.020892
2,1959-03-31 23:59:59.999999999,unemp,5.800,-1.269892
3,1959-06-30 23:59:59.999999999,realgdp,2778.801,0.659837
4,1959-06-30 23:59:59.999999999,infl,2.340,1.079641
5,1959-06-30 23:59:59.999999999,unemp,5.100,0.071027
6,1959-09-30 23:59:59.999999999,realgdp,2775.488,0.455204
7,1959-09-30 23:59:59.999999999,infl,2.740,-1.714606
8,1959-09-30 23:59:59.999999999,unemp,5.300,0.079527
9,1959-12-31 23:59:59.999999999,realgdp,2785.204,-0.650964


In [73]:
# 遗漏value参数，会得到一个含有多层列的DataFrame
pivoted1 = ldata.pivot('date', 'item')
pivoted1[:5]

value                    value2            \
item                           infl   realgdp unemp      infl   realgdp   
date                                                                      
1959-03-31 23:59:59.999999999  0.00  2710.349   5.8  0.020892 -0.350004   
1959-06-30 23:59:59.999999999  2.34  2778.801   5.1  1.079641  0.659837   
1959-09-30 23:59:59.999999999  2.74  2775.488   5.3 -1.714606  0.455204   
1959-12-31 23:59:59.999999999  0.27  2785.204   5.6  0.540690 -0.650964   
1960-03-31 23:59:59.999999999  2.31  2847.699   5.2  0.411236  1.338170   

                                         
item                              unemp  
date                                     
1959-03-31 23:59:59.999999999 -1.269892  
1959-06-30 23:59:59.999999999  0.071027  
1959-09-30 23:59:59.999999999  0.079527  
1959-12-31 23:59:59.999999999 -0.690465  
1960-03-31 23:59:59.999999999  1.379807

In [74]:
pivoted1['value'][:5]

item,infl,realgdp,unemp
date,,,
1959-03-31 23:59:59.999999999,0.00,2710.349,5.8
1959-06-30 23:59:59.999999999,2.34,2778.801,5.1
1959-09-30 23:59:59.999999999,2.74,2775.488,5.3
1959-12-31 23:59:59.999999999,0.27,2785.204,5.6
1960-03-31 23:59:59.999999999,2.31,2847.699,5.2


In [75]:
# pivot等价于使用set_index创建分层索引，然后调用unstack
unstacked = ldata.set_index(['date', 'item']).unstack('item')

In [76]:
unstacked[:5]

value                    value2            \
item                           infl   realgdp unemp      infl   realgdp   
date                                                                      
1959-03-31 23:59:59.999999999  0.00  2710.349   5.8  0.020892 -0.350004   
1959-06-30 23:59:59.999999999  2.34  2778.801   5.1  1.079641  0.659837   
1959-09-30 23:59:59.999999999  2.74  2775.488   5.3 -1.714606  0.455204   
1959-12-31 23:59:59.999999999  0.27  2785.204   5.6  0.540690 -0.650964   
1960-03-31 23:59:59.999999999  2.31  2847.699   5.2  0.411236  1.338170   

                                         
item                              unemp  
date                                     
1959-03-31 23:59:59.999999999 -1.269892  
1959-06-30 23:59:59.999999999  0.071027  
1959-09-30 23:59:59.999999999  0.079527  
1959-12-31 23:59:59.999999999 -0.690465  
1960-03-31 23:59:59.999999999  1.379807

### 8.3.3 将"宽"透视为"长"

In [77]:
# pivot的反向操作是pandas.melt。将一列变换为新DataFrame中的多列不同，
# 它将多列合并成一列，产生一个新的DataFrame，其长度比输入长度更长
df = pd.DataFrame({'key': ['foo', 'bar', 'baz'],
                   'A': [1, 2, 3],
                   'B': [4, 5, 6],
                   'C': [7, 8, 9]})
df

,key,A,B,C
0,foo,1,4,7
1,bar,2,5,8
2,baz,3,6,9


In [78]:
# key列可以作为分组指标，其他列均为数据值，但使用pandas.melt时，指定那些列是分组指标
melted = pd.melt(df, ['key'])
melted

,key,variable,value
0,foo,A,1
1,bar,A,2
2,baz,A,3
3,foo,B,4
4,bar,B,5
5,baz,B,6
6,foo,C,7
7,bar,C,8
8,baz,C,9


In [79]:
# 还原成原始的数据
melted.pivot('key', 'variable')

value      
variable     A  B  C
key                 
bar          2  5  8
baz          3  6  9
foo          1  4  7

In [80]:
# 也可以执行列的子集作为值列
pd.melt(df, id_vars=['key'], value_vars=['A', 'B'])

,key,variable,value
0,foo,A,1
1,bar,A,2
2,baz,A,3
3,foo,B,4
4,bar,B,5
5,baz,B,6


In [81]:
# 也可以不指定任何分组指标
pd.melt(df, value_vars=['A','B','C'])

,variable,value
0,A,1
1,A,2
2,A,3
3,B,4
4,B,5
5,B,6
6,C,7
7,C,8
8,C,9
